In [10]:
import numpy as np
import pandas as pd

In [11]:
class CreateSampleDataset():
    def __init__(self, n_teachers=1000, school_teacher_ratio=1.2, teacher_per_school=15.,
                 std_dev_teacher_per_school=3., min_teachers_per_school=5, p_rural=0.3, p_car=0.7, p_experienced=0.2):
        self.n_teachers = n_teachers
        self.n_schools = int(n_teachers * school_teacher_ratio / teacher_per_school)
        self.teacher_per_school = teacher_per_school
        self.std_dev_teacher_per_school = std_dev_teacher_per_school
        self.min_teachers_per_school = min_teachers_per_school
        self.p_rural = p_rural
        self.p_car = p_car
        self.p_experienced = p_experienced

    def _createTeachers(self):
        x_location = np.random.uniform(0., 1., self.n_teachers)
        y_location = np.random.uniform(0., 1., self.n_teachers)
        rural = np.random.choice([0, 1], p=[1 - self.p_rural, self.p_rural], size=self.n_teachers)
        car = np.random.choice([0, 1], p=[1 - self.p_car, self.p_car], size=self.n_teachers)
        experienced = np.random.choice([0, 1], p=[1 - self.p_experienced, self.p_experienced], size=self.n_teachers)
        teachers = {'x_location': x_location,
                    'y_location': y_location,
                    'rural': rural,
                    'car': car,
                    'experienced': experienced,
                    }

        return teachers

    def _createSchools(self):
        x_location = np.random.uniform(0., 1., self.n_schools)
        y_location = np.random.uniform(0., 1., self.n_schools)
        rural = np.random.choice([0, 1], p=[1 - self.p_rural, self.p_rural], size=self.n_schools)
        n_teachers = np.random.normal(loc=self.teacher_per_school, scale=self.std_dev_teacher_per_school, size=self.n_schools).astype(int)
        n_teachers_clipped = np.clip(n_teachers, a_min=self.min_teachers_per_school, a_max=None, out=n_teachers)
        schools = {'x_location': x_location,
                   'y_location': y_location,
                   'rural': rural,
                   'n_teachers': n_teachers_clipped,
                   }

        return schools

    def make_dataset(self):
        teachers = pd.DataFrame(self._createTeachers())
        schools = pd.DataFrame(self._createSchools())

        return teachers, schools

SyntaxError: positional argument follows keyword argument (<ipython-input-11-9cdcb1684ae7>, line 33)

In [12]:
DatasetSampler = CreateSampleDataset()

In [13]:
teachers, schools = DatasetSampler.make_dataset()

In [8]:
teachers.head()

,x_location,y_location,rural,car,experienced
0,0.694269,0.613895,0,0,0
1,0.756622,0.045317,0,0,0
2,0.751328,0.000510,0,0,0
3,0.076229,0.104822,1,1,0
4,0.446564,0.204482,0,0,0


In [9]:
schools.head()

,x_location,y_location,rural,n_teachers
0,0.846666,0.244801,1,16
1,0.448977,0.131370,0,10
2,0.484343,0.205962,1,16
3,0.613760,0.085730,1,14
4,0.850929,0.455456,0,15
